<a href="https://colab.research.google.com/github/MsMore/Vaani/blob/main/finetuning2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', )

Mounted at /content/drive


In [ ]:
import numpy as np
from keras import utils, callbacks
from keras.models import Sequential
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split, StratifiedKFold
#from keras.preprocessing.image import load_img, img_to_array
from skimage.transform import resize
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_directory = "/content/drive/MyDrive/Img/Train"
test_directory = "/content/drive/MyDrive/Img/Validation"

In [ ]:
from keras.applications import VGG16
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(128,128, 3))

58892288/58889256 [==============================] - 0s 0us/step


In [ ]:
# Freeze four convolution blocks
for layer in vgg_model.layers[:15]:
    layer.trainable = False
# Make sure you have frozen the correct layers
for i, layer in enumerate(vgg_model.layers):
    print(i, layer.name, layer.trainable)

0 input_1 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 False
12 block4_conv2 False
13 block4_conv3 False
14 block4_pool False
15 block5_conv1 True
16 block5_conv2 True
17 block5_conv3 True
18 block5_pool True


In [ ]:
x = vgg_model.output
x = Flatten()(x) # Flatten dimensions to for use in FC layers
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x) # Dropout layer to reduce overfitting
x = Dense(256, activation='relu')(x)
x = Dense(29, activation='softmax')(x) # Softmax for multiclass
transfer_model = Model(inputs=vgg_model.input, outputs=x)

In [ ]:
from keras.callbacks import ReduceLROnPlateau
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
lr_reduce = ReduceLROnPlateau(monitor='val_accuracy', factor=0.6, patience=8, verbose=1, mode='max', min_lr=5e-5)
checkpoint = ModelCheckpoint('vgg16_finetune.h15', monitor= 'val_accuracy', mode= 'max', save_best_only = True, verbose= 1)

In [ ]:
gen = ImageDataGenerator(rescale=1./255, validation_split=0.1) 
train = gen.flow_from_directory(train_directory, target_size=(128, 128), subset="training")
val = gen.flow_from_directory(test_directory, target_size=(128, 128), subset="validation")

Found 9135 images belonging to 29 classes.
Found 290 images belonging to 29 classes.


In [ ]:
from tensorflow.keras import layers, models, Model, optimizers
learning_rate= 5e-5
transfer_model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=learning_rate), metrics=["accuracy"])
history = transfer_model.fit(train, batch_size = 1, epochs=25, validation_data=val, callbacks=[lr_reduce,checkpoint])

Epoch 1/25
286/286 [==============================] - 41s 83ms/step - loss: 1.4180 - accuracy: 0.6330 - val_loss: 5.5736e-04 - val_accuracy: 1.0000

Epoch 00001: val_accuracy improved from -inf to 1.00000, saving model to vgg16_finetune.h15
INFO:tensorflow:Assets written to: vgg16_finetune.h15/assets
Epoch 2/25
286/286 [==============================] - 20s 71ms/step - loss: 0.0141 - accuracy: 0.9973 - val_loss: 1.6007e-04 - val_accuracy: 1.0000

Epoch 00002: val_accuracy did not improve from 1.00000
Epoch 3/25
286/286 [==============================] - 20s 71ms/step - loss: 0.0051 - accuracy: 0.9990 - val_loss: 6.6653e-04 - val_accuracy: 1.0000

Epoch 00003: val_accuracy did not improve from 1.00000
Epoch 4/25
286/286 [==============================] - 20s 71ms/step - loss: 0.0069 - accuracy: 0.9987 - val_loss: 1.0961e-05 - val_accuracy: 1.0000

Epoch 00004: val_accuracy did not improve from 1.00000
Epoch 5/25
286/286 [==============================] - 20s 71ms/step - loss: 0.0017 - a

In [ ]:
for layer in vgg_model.layers[:15]:
  layer.trainable = False
x = vgg_model.output
x = Flatten()(x) # Flatten dimensions to for use in FC layers
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x) # Dropout layer to reduce overfitting
x = Dense(256, activation='relu')(x)
x = Dense(29, activation='softmax')(x) # Softmax for multiclass
transfer_model = Model(inputs=vgg_model.input, outputs=x)
for i, layer in enumerate(transfer_model.layers):
  print(i, layer.name, layer.trainable)

0 input_1 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 False
12 block4_conv2 False
13 block4_conv3 False
14 block4_pool False
15 block5_conv1 True
16 block5_conv2 True
17 block5_conv3 True
18 block5_pool True
19 flatten_2 True
20 dense_6 True
21 dropout_2 True
22 dense_7 True
23 dense_8 True


In [ ]:
#Augment images
#rain_datagen = ImageDataGenerator(zoom_range=0.2, rotation_range=30, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2)
#Fit augmentation to training images
#train_generator = train_datagen.flow(train,batch_size=1)
#Compile model
transfer_model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])
#Fit model
history = transfer_model.fit_generator(train, validation_data=val, epochs=50, shuffle=True, callbacks=[lr_reduce],verbose=1)

In [ ]:
transfer_model.save("vggRA.h5")

NameError: ignored

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(transfer_model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpvpogwcth/assets


In [ ]:
import pathlib
tflite_models_dir = pathlib.Path("/tmp/mnist_tflite_models/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)
tflite_model_file = tflite_models_dir/"mnist_model.tflite"
tflite_model_file.write_bytes(tflite_model)

76202716

In [ ]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]

In [ ]:
tflite_fp16_model = converter.convert()
tflite_model_fp16_file = tflite_models_dir/"mnist_model_quant_f16.tflite"
tflite_model_fp16_file.write_bytes(tflite_fp16_model)

INFO:tensorflow:Assets written to: /tmp/tmpqbpme8gj/assets


INFO:tensorflow:Assets written to: /tmp/tmpqbpme8gj/assets


38110688

In [ ]:
import keras
model = keras.models.load_model("/content/drive/MyDrive/vggRA.h5")

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
def prepare_image(file):
    img_path = '/content/drive/MyDrive/Img/vggTest/'
    img = image.load_img(img_path + file, target_size=(128, 128))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return tf.keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)

In [ ]:
for i in range(1,59):  
  preprocessed_image = prepare_image('Test ({}).jpg'.format(i))
  predictions = model.predict(preprocessed_image)
  predictions=np.argmax(predictions[0])
  val=predictions
  str1="ABCD-EFGHIJKLMN=OPQRS TUVWXYZ"
  print(str1[val])

In [ ]:
import keras
model = keras.models.load_model("/content/drive/MyDrive/vggRiddhi.h5")

In [ ]:
import cv2
import numpy

cap = cv2.VideoCapture(0)
while True:
    cv2.resize(cap,(128,128,3))
    ret, frame = cap.read()
    image = frame
    #cv2.imread(frame)
    #cv2.resize(frame,(128,128))
    #image_np = np.array(frame)
    
    img_array = image.img_to_array(frame)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)

    predictions = model.predict(preprocessed_image)
    predictions=np.argmax(predictions[0])
    val=predictions
    str1="ABCD-EFGHIJKLMN=OPQRS TUVWXYZ"
    print(str1[val])

    cv2.imshow('object detection',  cv2.resize(image, (800, 600)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        break
cap.release()